In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import random
from tqdm import tqdm 

def collect_walker_data(num_walkers=10):
    all_data = []
    for i in range(num_walkers):
        filename = f"PATH/TO/COLVAR.{i}"
        try:
            with open(filename, 'r') as f:
                header = ''
                for line in f:
                    if line.startswith('#!'):
                        header = line.strip()
                        break
            
            if header:
                columns = header.replace('#! FIELDS', '').split()
            else:
                columns = ['time', 'A100', 'opes.bias', 'opes.rct', 'opes.zed']
            
            data = np.loadtxt(filename, comments=['#', '@'])
            
    
            time_idx = columns.index('time') if 'time' in columns else 0
            a100_idx = columns.index('A100') if 'A100' in columns else 1
            bias_idx = columns.index('opes.bias') if 'opes.bias' in columns else 2
            rct_idx = columns.index('opes.rct') if 'opes.rct' in columns else 3
            zed_idx = columns.index('opes.zed') if 'opes.zed' in columns else 4
            
            extracted_data = np.column_stack((
                data[:, time_idx],
                data[:, a100_idx],
                data[:, bias_idx],
                data[:, rct_idx],
                data[:, zed_idx]
            ))
            
            all_data.append(extracted_data)
            print(f"Successfully loaded data from {filename}: {len(extracted_data)} frames")
        except Exception as e:
            print(f"Warning: Could not load data for walker {i}: {e}")
    
    return np.vstack(all_data) if all_data else None


def reweight_data(data):
    A100_values = data[:, 1]  
    bias_values = data[:, 2]  
    rct_values = data[:, 3]  
    
    # Calculate weights
    beta = 1/(0.00831446*303.15)  # 1/kBT in mol/kJ
    weights = np.exp(beta * bias_values)
    
    # Normalize weights
    weights = weights / np.sum(weights)
    
    return A100_values, weights

def smooth_free_energy(values, weights, temp=303.15, grid_points=10000):
    bandwidth = 0.005 * np.std(values)

    kde = stats.gaussian_kde(values, weights=weights, bw_method=bandwidth)
    
    margin = 0.05 * (max(values) - min(values))
    grid = np.linspace(min(values) - margin, max(values) + margin, grid_points)
    
    pdf = kde(grid)
    
    # Free energy = -kBT * ln(P)
    kBT = 0.00831446 * temp 
    free_energy = -kBT * np.log(pdf + 1e-10)
    
    free_energy = free_energy - np.min(free_energy)
    
    return grid, free_energy


def bootstrap_free_energy_with_saving(values, weights, n_bootstrap=100, temp=303.15, grid_points=10000, 
                                      save_samples=True, bootstrap_file='A100_bootstrap_samples.npz',
                                      save_original=True, original_file='A100_original_data.npz'):
    """
    Perform bootstrap resampling to estimate standard error of free energy and save all samples.
    
    Parameters:
    -----------
    values : array
        The A100 values
    weights : array
        The weights from reweighting
    n_bootstrap : int
        Number of bootstrap samples
    temp : float
        Temperature in Kelvin
    grid_points : int
        Number of points for KDE evaluation
    save_samples : bool
        Whether to save all bootstrap samples
    bootstrap_file : str
        Filename to save bootstrap samples
    save_original : bool
        Whether to save the original A100 values and weights
    original_file : str
        Filename to save original data
        
    Returns:
    --------
    grid : array
        Grid points for x-axis
    fe_mean : array
        Mean free energy at each grid point
    fe_std : array
        Standard error at each grid point
    """
    if save_original:
        np.savez(original_file, 
                 a100_values=values, 
                 weights=weights,
                 description="Original A100 values and weights from reweighting")
        print(f"Original A100 values and weights saved to {original_file}")
    
    margin = 0.05 * (max(values) - min(values))
    grid = np.linspace(min(values) - margin, max(values) + margin, grid_points)
    
    bootstrap_results = []

    n_samples = len(values)
    print(f"Performing {n_bootstrap} bootstrap samples...")
    
    for i in tqdm(range(n_bootstrap)):
        indices = np.random.choice(range(n_samples), size=n_samples, replace=True)
        bootstrap_values = values[indices]
        bootstrap_weights = weights[indices]
        bootstrap_weights = bootstrap_weights / np.sum(bootstrap_weights)
        #KDE
        try:
            bandwidth = 0.005 * np.std(bootstrap_values)
            kde = stats.gaussian_kde(bootstrap_values, weights=bootstrap_weights, bw_method=bandwidth)
            
            # Evaluate KDE on the same grid
            pdf = kde(grid)
            
            # Calculate free energy
            kBT = 0.00831446 * temp  
            free_energy = -kBT * np.log(pdf + 1e-10)
            free_energy = free_energy - np.min(free_energy)
            bootstrap_results.append(free_energy)
        except Exception as e:
            print(f"Warning: Bootstrap sample {i} failed: {e}")
    
    bootstrap_results = np.array(bootstrap_results)
    
    # Calculate mean and standard deviation across bootstrap samples
    fe_mean = np.mean(bootstrap_results, axis=0)
    fe_std = np.std(bootstrap_results, axis=0)
    
    if save_samples:
        np.savez(bootstrap_file,
                 grid=grid,
                 bootstrap_samples=bootstrap_results,
                 fe_mean=fe_mean,
                 fe_std=fe_std,
                 metadata={
                     'n_bootstrap': n_bootstrap,
                     'temperature': temp,
                     'grid_points': grid_points,
                     'date': np.datetime64('now')
                 })
        print(f"All bootstrap samples saved to {bootstrap_file}")
    
    return grid, fe_mean, fe_std

def generate_free_energy_landscape_with_saved_samples():
    data = collect_walker_data()
    if data is None:
        return None
    A100_values, weights = reweight_data(data)
    grid_boot, fe_mean, fe_std = bootstrap_free_energy_with_saving(
        A100_values, weights, 
        save_samples=True, 
        bootstrap_file='A100_bootstrap_samples.npz',
        save_original=True,
        original_file='A100_original_data.npz'
    )
    
    plt.figure(figsize=(12, 8))
    
    plt.plot(grid_boot, fe_mean, '-', color='blue', label='Mean Free Energy')
    plt.fill_between(grid_boot, fe_mean - fe_std, fe_mean + fe_std, 
                     alpha=0.3, color='blue', label='±1 Standard Error')
    
    plt.xlabel('A100 score')
    plt.ylabel('Free Energy (kJ/mol)')
    plt.legend()
    plt.title('Free Energy Landscape of A100 with Bootstrapped Error Estimation')
    plt.savefig('A100_free_energy_with_error.png', dpi=300)
    plt.show()
    
    np.savetxt('A100_free_energy_with_error.dat', 
               np.column_stack((grid_boot, fe_mean, fe_std)),
               header='A100 Free_Energy(kJ/mol) StdError(kJ/mol)')
    
    return grid_boot, fe_mean, fe_std


if __name__ == "__main__":
    generate_free_energy_landscape_with_saved_samples()

In [ ]:
import numpy as np
def get_standard_error_at_a100(a100_value, bootstrap_file='A100_bootstrap_samples.npz'):
    """
    Calculate the standard error at a specified A100 value.
    
    Parameters:
    -----------
    a100_value : float
        The A100 value to get the standard error for
    bootstrap_file : str
        File containing the bootstrap samples
        
    Returns:
    --------
    se : float
        Standard error at the specified A100 value
    fe : float
        Free energy at the specified A100 value
    """
    bootstrap_data = np.load(bootstrap_file, allow_pickle=True)
    grid = bootstrap_data['grid']
    fe_mean = bootstrap_data['fe_mean']
    fe_std = bootstrap_data['fe_std']
    
    #Closest grid point
    idx = np.abs(grid - a100_value).argmin()
    
    # Get the standard error and free energy at this point
    se = fe_std[idx]
    fe = fe_mean[idx]
    
    print(f"At A100 = {a100_value:.4f} (closest grid point: {grid[idx]:.4f}):")
    print(f"Free Energy = {fe:.4f} kJ/mol")
    print(f"Standard Error = {se:.4f} kJ/mol")
    
    return se, fe

def find_energy_minimum_and_maximum(bootstrap_file='A100_bootstrap_samples.npz'):
    """
    Locate the energy minimum (global) and maximum (highest perceived barrier)
    and report their values with standard errors.
    
    Parameters:
    -----------
    bootstrap_file : str
        File containing the bootstrap samples
        
    Returns:
    --------
    min_data : tuple
        (A100_at_min, min_energy, min_se)
    max_data : tuple
        (A100_at_max, max_energy, max_se)
    """
    bootstrap_data = np.load(bootstrap_file, allow_pickle=True)
    grid = bootstrap_data['grid']
    fe_mean = bootstrap_data['fe_mean']
    fe_std = bootstrap_data['fe_std']
    
    # Find the minimum energy point
    min_idx = np.argmin(fe_mean)
    min_a100 = grid[min_idx]
    min_energy = fe_mean[min_idx]
    min_se = fe_std[min_idx]
    
    # Find the maximum energy point 
    max_idx = np.argmax(fe_mean)
    max_a100 = grid[max_idx]
    max_energy = fe_mean[max_idx]
    max_se = fe_std[max_idx]
    
    print(f"Energy Minimum:")
    print(f"A100 = {min_a100:.4f}")
    print(f"Free Energy = {min_energy:.4f} kJ/mol (This should be ~0)")
    print(f"Standard Error = {min_se:.4f} kJ/mol")
    print()
    print(f"Highest Energy Barrier:")
    print(f"A100 = {max_a100:.4f}")
    print(f"Free Energy = {max_energy:.4f} kJ/mol")
    print(f"Standard Error = {max_se:.4f} kJ/mol")
    
    return (min_a100, min_energy, min_se), (max_a100, max_energy, max_se)

min_data, max_data = find_energy_minimum_and_maximum()

se, fe = get_standard_error_at_a100(0)  # For quick assessment at given A100